In [ ]:
import torch

from models.clipseg import CLIPDensePredT
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt

# load model
model = CLIPDensePredT(version='ViT-B/16', reduce_dim=64)
model.eval();

# non-strict, because we only stored decoder weights (not CLIP weights)
model.load_state_dict(torch.load('weights.pth', map_location=torch.device('cpu')), strict=False)

Load and normalize `example_image.jpg`.

In [ ]:
# load and normalize image
input_image = Image.open('example_image.jpg')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((352, 352)),
])
img = transform(input_image).unsqueeze(0)

Predict and visualize

In [ ]:
# predict
with torch.no_grad():
    preds = model(img.repeat(4,1,1,1), ['a glass', 'cutlery', 'wood', 'a jar'])[0]

# visualize prediction
_, ax = plt.subplots(1, 5)
[a.axis('off') for a in ax.flatten()]
ax[0].imshow(input_image)
[ax[i+1].imshow(torch.sigmoid(preds[i][0])) for i in range(4)]